## Austin 311 Data Cleanup
-----
- A .csv file of Austin 311 calls was retrieved from the City of Austin data website. https://data.austintexas.gov/
- Donna Dietrich did the data cleaning and Tom assisted with the formatting of the df. 
- All members agreed on which columns to keep for the analysis.  

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

### The "austin_311_service_request.csv" file imported, read and the necessary columns were kept. 

In [2]:
#Read the csv file that was downloaded from the City of Austin: "https://data.austintexas.gov/"
austin_311_df = pd.read_csv('data/austin_311_service_requests.csv',low_memory=False)
austin_311_df.head()

,city,close_date,complaint_description,complaint_type,council_district_code,county,created_date,incident_address,incident_zip,last_update_date,...,map_tile,owning_department,source,state_plane_x_coordinate,state_plane_y_coordinate,status,status_change_date,street_name,street_number,unique_key
0,AUSTIN,2015-01-20 10:41:48,Loose Dog,ACLONAG,3.0,TRAVIS,2015-01-20 10:41:48,1305 CORIANDER DR AUSTIN TX,78741.0,2015-01-27 12:51:06,...,ML18,Animal Services Office,Phone,3131141.68710771,1.005532e+07,Duplicate (closed),2015-01-20 10:41:48,CORIANDER,1305.0,15-00012439
1,AUSTIN,2016-06-01 10:58:13,Loose Dog,ACLONAG,7.0,TRAVIS,2016-05-22 10:00:49,3104 STONEWAY DR AUSTIN TX 78757,78757.0,2016-06-01 09:58:30,...,MJ29,Animal Services Office,Phone,3112134.24930453,1.010083e+07,Closed,2016-06-01 10:58:13,STONEWAY,3104.0,16-00123310
2,AUSTIN,2014-01-06 19:30:00,Loose Animal (not dog),ACLOANIM,2.0,TRAVIS,2014-01-06 18:04:00,4100 SMITH SCHOOL RD AUSTIN TX,78744.0,2014-01-06 18:32:00,...,MK16,Animal Services Office,Phone,3124825.4024878,1.004597e+07,Closed,2014-01-06 19:30:00,SMITH SCHOOL,4100.0,14-00003554
3,AUSTIN,2014-11-19 10:46:48,Austin Code - Request Code Officer,CODECOMP,7.0,TRAVIS,2014-11-15 15:43:41,12710 PALFREY DR AUSTIN TX 78727,78727.0,2014-11-19 10:46:48,...,ML35,Austin Code Department,Web,3124206.25,1.012812e+07,Closed,2014-11-19 10:46:48,PALFREY,12710.0,14-00247505
4,AUSTIN,2014-08-25 21:21:00,Animal - Proper Care,ACPROPER,1.0,TRAVIS,2014-07-28 17:33:00,6703 TULANE DR AUSTIN TX,78723.0,2014-08-25 20:23:00,...,MM26,Animal Services Office,Phone,3137129.998,1.008721e+07,Closed,2014-08-25 21:21:00,TULANE,6703.0,14-00156342


In [3]:
austin_311_df.shape

(463540, 24)

In [5]:
austin_311_df.columns

Index(['city', 'close_date', 'complaint_description', 'complaint_type',
       'council_district_code', 'county', 'created_date', 'incident_address',
       'incident_zip', 'last_update_date', 'latitude', 'location', 'longitude',
       'map_page', 'map_tile', 'owning_department', 'source',
       'state_plane_x_coordinate', 'state_plane_y_coordinate', 'status',
       'status_change_date', 'street_name', 'street_number', 'unique_key'],
      dtype='object')

In [6]:
#Keep ['city', 'complaint_description', 'complaint_type', 'county', 'created_date', 
#'incident_zip', 'latitude', 'longitude', 'owning_department']

Austin_311_df = austin_311_df[["city", "county", "incident_zip", 
                            "created_date", "owning_department", 
                            "complaint_description", "complaint_type",
                           "latitude","longitude"]]

#Clean up the columns and put them in order.
Austin_311_df = Austin_311_df.rename(columns={"city":"City", "county":"County","incident_zip":"Incident Zip",
                                      "created_date":"Date", "owning_department":"Department",
                                      "complaint_description":"Complaint Description","complaint_type":"Type of Complaint",
                                       "latitude":"Lat", "longitude": "Lon"})
Austin_311_df.head()

,City,County,Incident Zip,Date,Department,Complaint Description,Type of Complaint,Lat,Lon
0,AUSTIN,TRAVIS,78741.0,2015-01-20 10:41:48,Animal Services Office,Loose Dog,ACLONAG,30.224549,-97.690675
1,AUSTIN,TRAVIS,78757.0,2016-05-22 10:00:49,Animal Services Office,Loose Dog,ACLONAG,30.350881,-97.747492
2,AUSTIN,TRAVIS,78744.0,2014-01-06 18:04:00,Animal Services Office,Loose Animal (not dog),ACLOANIM,30.199263,-97.711366
3,AUSTIN,TRAVIS,78727.0,2014-11-15 15:43:41,Austin Code Department,Austin Code - Request Code Officer,CODECOMP,30.425112,-97.707188
4,AUSTIN,TRAVIS,78723.0,2014-07-28 17:33:00,Animal Services Office,Animal - Proper Care,ACPROPER,30.311821,-97.669302


In [7]:
Austin_311_df.shape

(463540, 9)

In [9]:
# Check to see if there are any null cells in the rows. 
Austin_311_df.isnull().sum()

City                      8558
County                   10793
Incident Zip              8577
Date                     10799
Department                4235
Complaint Description     4235
Type of Complaint         4235
Lat                       7999
Lon                       7999
dtype: int64

In [10]:
#Getting the counts that are not null 
Austin_311_df.count()

City                     454982
County                   452747
Incident Zip             454963
Date                     452741
Department               459305
Complaint Description    459305
Type of Complaint        459305
Lat                      455541
Lon                      455541
dtype: int64

In [11]:
#Replace the empty rows with 'NaN'.
Austin_311_df.replace('', np.nan, inplace=True)

#Drop rows with 'NaN'.
clean_Austin_311_df = Austin_311_df.dropna(how="any")
clean_Austin_311_df.head()

,City,County,Incident Zip,Date,Department,Complaint Description,Type of Complaint,Lat,Lon
0,AUSTIN,TRAVIS,78741.0,2015-01-20 10:41:48,Animal Services Office,Loose Dog,ACLONAG,30.224549,-97.690675
1,AUSTIN,TRAVIS,78757.0,2016-05-22 10:00:49,Animal Services Office,Loose Dog,ACLONAG,30.350881,-97.747492
2,AUSTIN,TRAVIS,78744.0,2014-01-06 18:04:00,Animal Services Office,Loose Animal (not dog),ACLOANIM,30.199263,-97.711366
3,AUSTIN,TRAVIS,78727.0,2014-11-15 15:43:41,Austin Code Department,Austin Code - Request Code Officer,CODECOMP,30.425112,-97.707188
4,AUSTIN,TRAVIS,78723.0,2014-07-28 17:33:00,Animal Services Office,Animal - Proper Care,ACPROPER,30.311821,-97.669302


In [27]:
# Check to see if there are any null cells in the rows. 
clean_Austin_311_df.isnull().sum()

City                     0
County                   0
Incident Zip             0
Date                     0
Department               0
Complaint Description    0
Type of Complaint        0
Lat                      0
Lon                      0
dtype: int64

In [28]:
#Getting the counts for each row.  
clean_Austin_311_df.count()

City                     445814
County                   445814
Incident Zip             445814
Date                     445814
Department               445814
Complaint Description    445814
Type of Complaint        445814
Lat                      445814
Lon                      445814
dtype: int64

In [29]:
# Check to see if what cities are in the df.
clean_Austin_311_df['City'].value_counts()

AUSTIN              434850
PFLUGERVILLE          2071
DEL VALLE             1747
AUSTIN 5 ETJ          1204
MANOR                 1087
Austin                 538
austin                 498
CEDAR PARK             437
OTHER                  365
LEANDER                294
ROUND ROCK             251
BEE CAVE               234
WEST LAKE HILLS        232
LAKEWAY                203
SUNSET VALLEY          200
MUSTANG RIDGE          183
ELGIN                  179
MANCHACA               117
TRAVIS                 111
SPICEWOOD              106
BUDA                    99
JONESTOWN               92
LAGO VISTA              90
CREEDMOOR               76
ROLLINGWOOD             75
WEBBERVILLE             72
BRIARCLIFF              58
DRIPPING SPRINGS        46
CEDAR CREEK             34
COUPLAND                33
                     ...  
KYLE                     3
HUTTO                    3
AUstin                   2
BEAR CREEK               2
Austin.                  2
Aus                      2
S

In [30]:
# Check to see if what counties are in the df. 
clean_Austin_311_df['County'].value_counts()

TRAVIS        432185
WILLIAMSON     13519
HAYS              93
BASTROP           17
Name: County, dtype: int64

### Fitlering out cities that aren't "Austin" and cities not in "Travis" county. 
-----
Since the majority of the 311 calls were in Travis county and in Austin we decided to focus on Austin, Travis county.  There were many misspellings of "Austin" that we chose to keep, which include: 'AUSTIN' and 'austin'.  Then we kept only these citys that were in Travis county. 

In [31]:
#Keep all cities in Travis county.

Austin_311_Travis = clean_Austin_311_df[clean_Austin_311_df.County.isin(['TRAVIS'])]
Austin_311_Travis['County'].value_counts()

TRAVIS    432185
Name: County, dtype: int64

In [37]:
Austin_311_Travis['City'].value_counts()

AUSTIN              421862
PFLUGERVILLE          2065
DEL VALLE             1744
AUSTIN 5 ETJ          1191
MANOR                 1087
Austin                 488
austin                 477
OTHER                  362
LEANDER                276
BEE CAVE               234
WEST LAKE HILLS        232
LAKEWAY                203
SUNSET VALLEY          200
CEDAR PARK             200
MUSTANG RIDGE          182
ELGIN                  179
TRAVIS                 108
SPICEWOOD              106
MANCHACA               102
JONESTOWN               92
LAGO VISTA              88
BUDA                    79
CREEDMOOR               76
ROLLINGWOOD             75
WEBBERVILLE             71
BRIARCLIFF              58
ROUND ROCK              46
CEDAR CREEK             34
COUPLAND                33
DRIPPING SPRINGS        28
                     ...  
MARBLE FALLS             5
AustiN                   4
Manchaca                 4
Pflugerville             3
aus                      3
HUTTO                    2
a

In [38]:
# Keep 'AUSTIN', 'Austin', and 'austin'. 

All_Austin_311 = Austin_311_Travis[Austin_311_Travis.City.isin(['AUSTIN', 'Austin', 'austin'])]

# Get the new counts
All_Austin_311.count()

City                     422827
County                   422827
Incident Zip             422827
Date                     422827
Department               422827
Complaint Description    422827
Type of Complaint        422827
Lat                      422827
Lon                      422827
dtype: int64

In [39]:
All_Austin_311['City'].value_counts()

AUSTIN    421862
Austin       488
austin       477
Name: City, dtype: int64

In [40]:
All_Austin_311.head()

,City,County,Incident Zip,Date,Department,Complaint Description,Type of Complaint,Lat,Lon
0,AUSTIN,TRAVIS,78741.0,2015-01-20 10:41:48,Animal Services Office,Loose Dog,ACLONAG,30.224549,-97.690675
1,AUSTIN,TRAVIS,78757.0,2016-05-22 10:00:49,Animal Services Office,Loose Dog,ACLONAG,30.350881,-97.747492
2,AUSTIN,TRAVIS,78744.0,2014-01-06 18:04:00,Animal Services Office,Loose Animal (not dog),ACLOANIM,30.199263,-97.711366
3,AUSTIN,TRAVIS,78727.0,2014-11-15 15:43:41,Austin Code Department,Austin Code - Request Code Officer,CODECOMP,30.425112,-97.707188
4,AUSTIN,TRAVIS,78723.0,2014-07-28 17:33:00,Animal Services Office,Animal - Proper Care,ACPROPER,30.311821,-97.669302


In [41]:
# Change the zip to an integer
All_Austin_311["Incident Zip"] = All_Austin_311["Incident Zip"].astype(int)

#Drop the City and County since we don't need them anymore. 
All_Austin_311.drop('City',axis=1, inplace=True)
All_Austin_311.drop('County',axis=1, inplace=True)

#Change the date to a datetime format so we can extract month and year for new columns.
All_Austin_311['Date'] = pd.to_datetime(All_Austin_311['Date'], format='%Y/%m/%d')

#Create columns for year and month. 
All_Austin_311['Year'] = All_Austin_311['Date'].dt.year
All_Austin_311['Month'] = All_Austin_311['Date'].dt.month

#Drop the Date since we don't need it anymore. 
All_Austin_311.drop('Date',axis=1, inplace=True)

/Users/tom/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/tom/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/tom/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/tom/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:

In [42]:
All_Austin_311.head()

,Incident Zip,Department,Complaint Description,Type of Complaint,Lat,Lon,Year,Month
0,78741,Animal Services Office,Loose Dog,ACLONAG,30.224549,-97.690675,2015,1
1,78757,Animal Services Office,Loose Dog,ACLONAG,30.350881,-97.747492,2016,5
2,78744,Animal Services Office,Loose Animal (not dog),ACLOANIM,30.199263,-97.711366,2014,1
3,78727,Austin Code Department,Austin Code - Request Code Officer,CODECOMP,30.425112,-97.707188,2014,11
4,78723,Animal Services Office,Animal - Proper Care,ACPROPER,30.311821,-97.669302,2014,7


In [43]:
#Save the clean df as .csv file
#All_Austin_311.to_csv('data/All_Austin_311.csv', encoding='utf-8', index=False)